# Dihedral symmetry of the Robinson form

**Adapted from**: Example 5.4 of [GP04]

[GP04] Gatermann, Karin and Parrilo, Pablo A.
*Symmetry groups, semidefinite programs, and sums of squares*.
Journal of Pure and Applied Algebra 192.1-3 (2004): 95-128.

Symmetry reduction is still a work in progress in SumOfSquares, so we include the following files that will be incorporated into SumOfSquares.jl once SymbolicWedderburn.jl is released:

In [1]:
using SumOfSquares
include(joinpath(dirname(dirname(pathof(SumOfSquares))), "examples", "symmetry.jl"))
include(joinpath(dirname(dirname(pathof(SumOfSquares))), "examples", "scaled_perm.jl"))

    Cloning git-repo `https://github.com/kalmarek/SymbolicWedderburn.jl`
   Updating git-repo `https://github.com/kalmarek/SymbolicWedderburn.jl`
   Updating registry at `~/.julia/registries/General`
  Resolving package versions...
Updating `~/work/SumOfSquares.jl/SumOfSquares.jl/docs/Project.toml`
  [858aa9a9] + SymbolicWedderburn v0.1.0 `https://github.com/kalmarek/SymbolicWedderburn.jl#bl/sos`
Updating `~/work/SumOfSquares.jl/SumOfSquares.jl/docs/Manifest.toml`
  [858aa9a9] + SymbolicWedderburn v0.1.0 `https://github.com/kalmarek/SymbolicWedderburn.jl#bl/sos`


We start by defining the Dihedral group of order 8.
This group is isomorphic to the following permutation group:

In [2]:
d = perm"(1, 2, 3, 4)"
c = perm"(1, 3)"
G = PermGroup([c, d])

Permutation group on 2 generators generated by
 (1,3)
 (1,2,3,4)

We could rely on this isomorphism to define this group.
However, in order to illustrate how to do symmetry reduction with a custom group,
we show in this example what should be implemented to define a new group.

In [3]:
struct DihedralElement <: GroupElem
    n::Int
    reflection::Bool
    id::Int
end
function PermutationGroups.order(el::DihedralElement)
    if el.reflection
        return 2
    else
        if iszero(el.id)
            return 1
        else
            return div(el.n, gcd(el.n, el.id))
        end
    end
end
Base.one(el::DihedralElement) = DihedralElement(el.n, false, 0)
function Base.inv(el::DihedralElement)
    if el.reflection || iszero(el.id)
        return el
    else
        return DihedralElement(el.n, false, el.n - el.id)
    end
end
function Base.:*(a::DihedralElement, b::DihedralElement)
    a.n == b.n || error("Cannot multiply elements from different Dihedral groups")
    id = mod(a.reflection ? a.id - b.id : a.id + b.id, a.n)
    return DihedralElement(a.n, a.reflection != b.reflection, id)
end
function PermutationGroups.mul!(::DihedralElement, a::DihedralElement, b::DihedralElement)
    return a * b
end
function Base.:^(el::DihedralElement, k::Integer)
    if el.reflection
        return iseven(k) ? one(el) : el
    else
        return DihedralElement(el.n, false, mod(el.id * k, el.n))
    end
end

Base.conj(a::DihedralElement, b::DihedralElement) = inv(b) * a * b
Base.:^(a::DihedralElement, b::DihedralElement) = conj(a, b)

struct DihedralGroup <: Group
    n::Int
end
Base.one(G::DihedralGroup) = DihedralElement(G.n, false, 0)
PermutationGroups.gens(G::DihedralGroup) = [DihedralElement(G.n, false, 1), DihedralElement(G.n, true, 0)]
PermutationGroups.order(::Type{T}, G::DihedralGroup) where {T} = convert(T, 2G.n)
function Base.iterate(G::DihedralGroup, prev::DihedralElement=DihedralElement(G.n, false, -1))
    if prev.id + 1 >= G.n
        if prev.reflection
            return nothing
        else
            next = DihedralElement(G.n, true, 0)
        end
    else
        next = DihedralElement(G.n, prev.reflection, prev.id + 1)
    end
    return next, next
end

The Robinson form is invariant under the following action of the Dihedral group on monomials:
The action of each element of the groups is to map the variables `x, y` to:

| id | rotation | reflection |
|----|----------|------------|
| 0  | x, y     | y, x       |
| 1  | -y, x    | -x, y      |
| 2  | -x, -y   | -y, -x     |
| 3  | y, -x    | x, -y      |

In [4]:
using DynamicPolynomials
@polyvar x y
function action(mono::MP.AbstractMonomial, el::DihedralElement)
    if iseven(el.reflection + el.id)
        var_x, var_y = x, y
    else
        var_x, var_y = y, x
    end
    sign_x = 1 <= el.id <= 2 ? -1 : 1
    sign_y = 2 <= el.id ? -1 : 1
    return MP.substitute(MP.Eval(), mono, [x, y] => [sign_x * var_x, sign_y * var_y])
end
function action(term::MP.AbstractTerm, el::DihedralElement)
    return MP.coefficient(term) * action(MP.monomial(term), el)
end
function action(poly::MP.AbstractPolynomial, el::DihedralElement)
    return MP.polynomial([action(term, el) for term in MP.terms(poly)])
end

poly = x^6 + y^6 - x^4 * y^2 - y^4 * x^2 - x^4 - y^4 - x^2 - y^2 + 3x^2 * y^2 + 1

x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1

We can verify that `poly` is indeed invariant under the action of each element of the group as follows.

In [5]:
G = DihedralGroup(4)
for g in G
    @show action(poly, g)
end

action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1


We can exploit this symmetry for reducing the problem using the `SymmetricIdeal` certificate as follows:

In [6]:
import CSDP
function solve(G)
    solver = CSDP.Optimizer
    model = Model(solver)
    @variable(model, t)
    @objective(model, Max, t)
    certificate = SymmetricIdeal(Certificate.MaxDegree(SOSCone(), MonomialBasis, maxdegree(poly)), G, action)
    con_ref = @constraint(model, poly - t in SOSCone(), ideal_certificate = certificate)
    optimize!(model)
    @show value(t)


    for g in gram_matrix(con_ref).sub_gram_matrices
        println(g.basis.polynomials)
    end
end
solve(G)

CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 7.62e-01 Pobj: -3.7013654e+00 Ad: 8.82e-01 Dobj:  1.4448006e+00 
Iter:  2 Ap: 7.89e-01 Pobj: -1.1744854e+01 Ad: 8.88e-01 Dobj:  7.0416913e+00 
Iter:  3 Ap: 8.76e-01 Pobj: -4.9005925e+00 Ad: 7.61e-01 Dobj:  3.4866723e+00 
Iter:  4 Ap: 8.50e-01 Pobj: -2.2686990e+00 Ad: 8.19e-01 Dobj:  4.5677896e-01 
Iter:  5 Ap: 8.06e-01 Pobj: -1.5023382e+00 Ad: 7.88e-01 Dobj: -4.4550461e-01 
Iter:  6 Ap: 7.53e-01 Pobj: -1.1112434e+00 Ad: 7.88e-01 Dobj: -7.3487497e-01 
Iter:  7 Ap: 8.78e-01 Pobj: -1.0096901e+00 Ad: 6.94e-01 Dobj: -8.4673473e-01 
Iter:  8 Ap: 7.28e-01 Pobj: -9.6560881e-01 Ad: 7.60e-01 Dobj: -8.9973797e-01 
Iter:  9 Ap: 8.15e-01 Pobj: -9.4894344e-01 Ad: 7.29e-01 Dobj: -9.2046926e-01 
Iter: 10 Ap: 1.00e+00 Pobj: -9.2795803e-01 Ad: 8.81e-01 Dobj: -9.2710946e-01 
Iter: 11 Ap: 8.20e-01 Pobj: -9.2929741e-01 Ad: 8.79e-01 Dobj: -9.3031738e-01 
Iter: 12 Ap: 1.00e+00 Pobj: -9.2915503e-01 Ad: 1.00e+

We notice that we indeed find `-3825/4096` and that symmetry was exploited.
In case the conjugacy classes are known, we can implement
`SymbolicWedderburn.conjugacy_classes_orbit` instead of `order` and `iterate`.
To show that these do not need to be implemented, we create a new dihedral group type
that do not implement these methods but that instead implement
`SymbolicWedderburn.conjugacy_classes_orbit`:

In [7]:
struct DihedralGroup2 <: Group
    n::Int
end
PermutationGroups.gens(G::DihedralGroup2) = [DihedralElement(G.n, false, 1), DihedralElement(G.n, true, 0)]
_orbit(cc::Vector{<:GroupElem}) = PermutationGroups.Orbit(cc, Dict(a => nothing for a in cc))
_orbit(el::GroupElem) = _orbit([el])
function SymbolicWedderburn.conjugacy_classes_orbit(d::DihedralGroup2)
    orbits = [_orbit(DihedralElement(d.n, false, 0))]
    for i in 1:div(d.n - 1, 2)
        push!(orbits, _orbit([
            DihedralElement(d.n, false, i),
            DihedralElement(d.n, false, d.n - i),
        ]))
    end
    if iseven(d.n)
        push!(orbits, _orbit(DihedralElement(d.n, false, div(d.n, 2))))
        push!(orbits, _orbit([
            DihedralElement(d.n, true, i) for i in 0:2:(d.n - 2)
        ]))
        push!(orbits, _orbit([
            DihedralElement(d.n, true, i) for i in 1:2:(d.n - 1)
        ]))
    else
        push!(orbits, _orbit([
            DihedralElement(d.n, true, i) for i in 0:(d.n - 1)
        ]))
    end
end

As we have not implemented the iterator over all the elements, we can iterate over all
conjugacy classes instead to verify that the polynomial is invariant under the group action.

In [8]:
G = DihedralGroup2(4)
for cc in SymbolicWedderburn.conjugacy_classes_orbit(G)
    for g in cc
        @show action(poly, g)
    end
end

solve(G)

action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
Stuck at edge of primal feasibility, giving up. 
Partial Success: SDP solved with reduced accuracy
Primal objective value: -9.3249138e-01 
Dual objective value: -9.3316417e-01 
Relative primal infeasibility: 2.47e-08 
Relative dual infeasibility: 9.37e-11 
Real Relative Gap: -2.35e-04 
XZ Relative Gap: 4.59e-10 
DIMACS error measures: 3.23e-08 0.00e+00 2.47e-10 0.00e+00 -2.35e-04 4.59e-10
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*